In [104]:
import gym 
from gym import envs
import numpy as np 
from time import sleep

In [105]:
env = gym.make('Taxi-v3')
env.reset() #returns random initial state
env.render() #renders the environment
print("State size ", env.observation_space.n) #represents all possible locations
print("Action size ", env.action_space.n) #in this case our actions are N,S,E,W,P/U,D/O 

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

State size  500
Action size  6


The objective of this policy evaluation is to converge to the true value function for a given policy. This will return the required value function.

In [106]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    # Start with a random (all 0) value function
    V = np.zeros(env.env.nS)
    while True:
        delta = 0  #delta = change in value of state from one iteration to next
       
        for state in range(env.env.nS):  #for all states
            val = 0  #initiate value as 0
            
            for action,act_prob in enumerate(policy[state]): #for all actions/action probabilities
                for prob,next_state,reward,done in env.env.P[state][action]:  #transition probabilities,state,rewards of each action
                    val += act_prob * prob * (reward + discount_factor * V[next_state])  #eqn to calculate
            delta = max(delta, np.abs(val-V[state]))
            V[state] = val
        if delta < theta:  #break if the change in value is less than the threshold of theta
            break
    return np.array(V) #value function

The below block includes the policy iteration and policy improvement part of the algorithm.
This will return a tuple, which is the optimal policy matrix and value function for each state.
We need a helper function that does one step lookahead to calculate the state-value function

In [107]:
def policy_iteration(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    def policy_improvement(state, V):
        A = np.zeros(env.env.nA)
        for a in range(env.env.nA):
            for prob, next_state, reward, done in env.env.P[state][a]:
                A[a] += prob * (reward + discount_factor * V[next_state])
        return A
    # Start with a random policy
    policy = np.ones([env.env.nS, env.env.nA]) / env.env.nA

    while True:
        # Implement this!
        curr_pol_val = policy_eval_fn(policy, env, discount_factor)  #eval current policy
        policy_stable = True  #check whether policy improved (set initially as true)
        for state in range(env.env.nS):  #for each state
            chosen_act = np.argmax(policy[state]) 
            act_values = policy_improvement(state,curr_pol_val)  #use one step lookahead to find action values
            best_act = np.argmax(act_values) #find the best action
            if chosen_act != best_act:
                policy_stable = False  #greedily find best action
            policy[state] = np.eye(env.env.nA)[best_act]  #update 
        if policy_stable:
            return policy, curr_pol_val #expected value of each action
    
    return policy, np.zeros(env.env.nS)

In [108]:
opt_policy = policy_iteration(env,policy_eval,discount_factor=0.99)
opt_policy[0]

array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.]])

The code block below enables us to see our optimal policy in action.

In [109]:
def view_policy(policy):
    curr_state = env.reset()
    counter = 0
    reward = None
    while reward != 20:
        state, reward, done, info = env.step(np.argmax(policy[0][curr_state])) 
        curr_state = state
        counter += 1
        env.env.s = curr_state
        print(f"")
        print(f"State:",state)
        env.render()
        print(f"Step:",counter)
        print(f"Reward:",reward)
     

Below shows optimal policy for thie iteration. The taxi picks up the passenger and drops them off without a detour.

In [110]:
view_policy(opt_policy)



State: 391
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
Step: 1
Reward: -1

State: 291
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
Step: 2
Reward: -1

State: 271
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
Step: 3
Reward: -1

State: 251
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
Step: 4
Reward: -1

State: 231
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
Step: 5
Reward: -1

State: 211
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
Step: 6
Reward: -1

State: 311
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
Step: 7
Reward: -1

State: 411
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
Step: 8
Reward: -1

Sta